In [1]:
import pandas as pd
import json
import hashlib
import time
import requests
import time
import cPickle as pickle
from tqdm import tqdm

In [166]:
df = pd.read_csv('final_final_feature_set.csv', encoding='ISO-8859-1')
df = df.fillna('')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35148 entries, 0 to 35147
Data columns (total 15 columns):
indicator_types.value                                           35148 non-null object
kill_chain_phases.kill_chain_phases.ordinality                  35148 non-null object
observable.object.properties.address_value.value                35148 non-null object
observable.object.properties.file_name.value                    35148 non-null object
observable.object.properties.hashes.simple_hash_value.value     35148 non-null object
observable.object.properties.hashes.type.value                  35148 non-null object
observable.object.properties.header.from.address_value.value    35148 non-null object
observable.object.properties.header.subject.value               35148 non-null object
observable.object.properties.size_in_bytes.value                35148 non-null object
observable.object.properties.value.value                        35148 non-null object
related_objects.properties.valu

In [167]:
url_score_column = []

def replace_url_with_score(url_list):
    url_scores = pickle.load(open('url_score_dict.pickle', 'rb'))
    
    for url in tqdm(url_list):
        try:
            url_score_column.append(float(url_scores[url]))
        except KeyError:
            url_score_column.append('')

replace_url_with_score(df['observable.object.properties.value.value'])

len(url_score_column)

100%|██████████| 35148/35148 [00:00<00:00, 251118.11it/s]


35148

In [168]:
ip_score_col = []

def replace_ip_with_score(ip_list):
    ip_scores = pickle.load(open('ip_scores.pickle', 'rb'))
    
    for ip in tqdm(ip_list):
        try:
            ip_score_col.append(float(ip_scores[ip]))
        except (KeyError, TypeError):
            ip_score_col.append('')            

replace_ip_with_score(df['observable.object.properties.address_value.value'])

len(ip_score_col)

100%|██████████| 35148/35148 [00:00<00:00, 374229.62it/s]


35148

In [169]:
df.insert(2, 'ip_score', ip_score_col)
df.insert(3, 'url_score', url_score_column)

In [170]:
df['file_type'] = df['observable.object.properties.file_name.value'].apply(lambda x : x.split('.')[-1])

In [171]:
df.columns

Index([                                       u'indicator_types.value',
                     u'kill_chain_phases.kill_chain_phases.ordinality',
                                                           u'ip_score',
                                                          u'url_score',
                   u'observable.object.properties.address_value.value',
                       u'observable.object.properties.file_name.value',
        u'observable.object.properties.hashes.simple_hash_value.value',
                     u'observable.object.properties.hashes.type.value',
       u'observable.object.properties.header.from.address_value.value',
                  u'observable.object.properties.header.subject.value',
                   u'observable.object.properties.size_in_bytes.value',
                           u'observable.object.properties.value.value',
                             u'related_objects.properties.value.value',
                                                              u'

In [172]:
unwanted_columns = ['observable.object.properties.address_value.value',
                    'observable.object.properties.hashes.simple_hash_value.value', 
                    'observable.object.properties.header.from.address_value.value',
                    'observable.object.properties.value.value',
                    'related_objects.properties.value.value',
                    'observable.object.properties.file_name.value',
                    'observable.object.properties.header.from.address_value.value',
                    'title']

df = df.drop(unwanted_columns, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35148 entries, 0 to 35147
Data columns (total 11 columns):
indicator_types.value                                35148 non-null object
kill_chain_phases.kill_chain_phases.ordinality       35148 non-null object
ip_score                                             35148 non-null object
url_score                                            35148 non-null object
observable.object.properties.hashes.type.value       35148 non-null object
observable.object.properties.header.subject.value    35148 non-null object
observable.object.properties.size_in_bytes.value     35148 non-null object
tlp_marking_color                                    35148 non-null object
target                                               35148 non-null object
hash_score                                           35148 non-null object
file_type                                            35148 non-null object
dtypes: object(11)
memory usage: 2.9+ MB


In [173]:
cat_columns = ['observable.object.properties.hashes.type.value', 
               'observable.object.properties.header.subject.value', 
               'file_type',
               'indicator_types.value',
               'tlp_marking_color',
               'target'
              ]

df = df.replace('', 0.0)
# Converting object datatype to category datatype
for col in list(cat_columns):
    df[col] = df[col].astype('category')

# Assigning a code to each category for the model to learn it easily
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df.head()

,indicator_types.value,kill_chain_phases.kill_chain_phases.ordinality,ip_score,url_score,observable.object.properties.hashes.type.value,observable.object.properties.header.subject.value,observable.object.properties.size_in_bytes.value,tlp_marking_color,target,hash_score,file_type
0,6,0.0,0.00000,0.0,0,0,0.0,0,5,0.0,0
1,6,0.0,1.00000,0.0,0,0,0.0,0,5,0.0,0
2,6,0.0,0.00000,0.0,0,0,0.0,0,5,0.0,0
3,6,0.0,0.00000,0.0,0,0,0.0,0,5,0.0,0
4,6,0.0,0.51408,0.0,0,0,0.0,0,5,0.0,0


In [193]:
df.to_csv('normalized_feature_set.csv', index=False, encoding='utf-8')